refer to:
https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_train.columns

In [ ]:
print(df_train.shape)
print(df_train.columns.shape)
df_train['SalePrice'].describe()

In [ ]:
sns.distplot(df_train['SalePrice'])

In [ ]:
for a in 'skew', 'kurt':
    print('{}: {:f}'.format(a, getattr(df_train['SalePrice'], a)()))

In [ ]:
def scatter2d(df, name_x, name_y, ylim=None):
    data = pd.concat([df_train[name_y], df_train[name_x]], axis=1)
    data.plot.scatter(x=name_x, y=name_y, ylim=ylim)

In [ ]:
scatter2d(df_train, 'GrLivArea', 'SalePrice', (0, 800000))

In [ ]:
scatter2d(df_train, 'TotalBsmtSF', 'SalePrice', ylim=(0, 800000))

In [ ]:
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000)

In [ ]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90)

In [ ]:
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
k = 10
corrmat.nlargest(k, 'SalePrice')

In [ ]:
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cols

In [ ]:
df_train[cols]

In [ ]:
df_train[cols].values

In [ ]:
cm = np.corrcoef(df_train[cols].values.T)
cm

In [ ]:
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, fmt='.2f', annot_kws={'size': 10},
 yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size=2.5)
plt.show();

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index, 1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max()

In [ ]:
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:, np.newaxis]);
low_range  = saleprice_scaled[saleprice_scaled[:, 0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:, 0].argsort()][-10:]
print("outer range (low) of the distribution:")
print(low_range)
print("\nouter range (high) of the distribution:")
print(high_range)